# Read SeaWIFs 8 day extracted data

this R notebook reads netcdf files from L3 Browser @ NASA Ocean Color https://oceancolor.gsfc.nasa.gov/l3/

the data is from 8-day running averages of PAR measured by SeaWIFs extracted from the Cariaco Region

In [1]:
library("ncdf4")
library("reshape2")

In [2]:
outputfile <- "DATA_processed/SeaWIFs_8day_extracted.csv"
if (file.exists(outputfile)) file.remove(outputfile)    # caution new data will be appended to this file if it already exists
print(outputfile)

[1] TRUE

[1] "DATA_processed/SeaWIFs_8day_extracted.csv"


## create bounding box & create list of file names

In [3]:
# list and remove objects
ls()
rm(list = ls())

# set the study area
latmax <- 10.78
latmin <- 10.35
lonmax <- -64.53
lonmin <- -64.9


lat = c(10.35,  10.78, 10.78, 10.35)
lon = c(-64.53,-64.53,-64.9, -64.9)

# latmax <- 10.6
# latmin <- 10.4
# lonmax <- -64.75
# lonmin <- -65.6
# latmax <- 10.8
# latmin <- 10.5
# lonmax <- -64.6
# lonmin <- -65.2

# create a list of files and indicate its length
f <- list.files("DATA_raw/SeaWIFs_PAR_extracted/.", pattern="*.L3m_8D_PAR.x_par.nc",full.names=F)
lf<-length(f)
print(head(f))

[1] "outputfile"

[1] "S19972491997256.L3m_8D_PAR.x_par.nc" "S19972571997264.L3m_8D_PAR.x_par.nc"
[3] "S19972651997272.L3m_8D_PAR.x_par.nc" "S19972731997280.L3m_8D_PAR.x_par.nc"
[5] "S19972811997288.L3m_8D_PAR.x_par.nc" "S19972891997296.L3m_8D_PAR.x_par.nc"


## show netcdf file structure

In [4]:
data<-nc_open(paste('DATA_raw/SeaWIFs_PAR_extracted/',f[1],sep=''))
print(data)
# extract data
nc_close(data)

File DATA_raw/SeaWIFs_PAR_extracted/S19972491997256.L3m_8D_PAR.x_par.nc (NC_FORMAT_NETCDF4):

     2 variables (excluding dimension variables):
        short par[lon,lat]   (Chunking: [23,26])  (Compression: shuffle,level 4)
            long_name: Photosynthetically Available Radiation, R. Frouin
            scale_factor: 0.0020000000949949
            add_offset: 65.5
            units: einstein m^-2 day^-1
            standard_name: surface_downwelling_photosynthetic_photon_flux_in_air
            _FillValue: -32767
            valid_min: -32750
            valid_max: 32250
            reference: Frouin, R., Ligner, D.W., and Gautier, C., 1989: A Simple analytical formula to compute clear sky total and photosynthetically available solar irradiance at the ocean surface. J. Geophys. Res., 94, 9731-9742.
            display_scale: linear
            display_min: 0
            display_max: 76.1999969482422
        unsigned byte palette[eightbitcolor,rgb]   (Contiguous storage)  

     4 

## define variable to extract : PAR

In [5]:
# variable
var<-"par"

## Open each individual file and extract PAR values within bounding box defined above

In [6]:
for (i in 1:lf) {
  # progress indicator
  print(paste("Processing file",i,"from",length(f),sep=" "))
  # open netCDF file
  data<-nc_open(paste('DATA_raw/SeaWIFs_PAR_extracted/',f[i],sep=''))
  # extract data
  lon<-ncvar_get(data,"lon")
  lat<-ncvar_get(data,"lat")
  value<-ncvar_get(data,var)
  unit<-ncatt_get(data,var,"units")$value
  # matrix to data.frame
  dimnames(value)<-list(lon=lon,lat=lat)
  dat.var<-melt(value,id="lon")
  # select data from the study area taking out missing data
  dat.varSAtmp<-subset(dat.var,lon<=lonmax & lon>=lonmin & lat<=latmax & lat>=latmin)
  # extract date information
  dateini<-ncatt_get(data,0,"time_coverage_start")$value
  dateend<-ncatt_get(data,0,"time_coverage_end")$value
  datemean<-mean(c(as.Date(dateend,"%Y-%m-%dT%H:%M:%OSZ"),as.Date(dateini,"%Y-%m-%dT%H:%M:%OSZ")))
  year<- substring(datemean,0,4) #1997 + ((i+6)/12)#
  month<- substring(datemean,6,7) #i+6 #
  # prepare final data set
  dat.varSA<-data.frame(rep(as.integer(year,nrow(dat.varSAtmp))),rep(as.integer(month,nrow(dat.varSAtmp))),
                        dat.varSAtmp,rep(unit,nrow(dat.varSAtmp)),rep(var,nrow(dat.varSAtmp)))
  names(dat.varSA)<-c("year","month","lon","lat","value","unit","var")
  # save csv file
  fe<-file.exists(paste('DATA_processed/','SeaWIFs_8day_extracted.csv',sep=''))
  write.table(dat.varSA,paste('DATA_processed/','SeaWIFs_8day_extracted.csv',sep=''),row.names=FALSE,col.names=!fe,sep=",",dec=".",append=fe)
  # close connection
  nc_close(data)
  # clean workspace
  rm(data,lon,lat,value,unit,dat.var,dat.varSAtmp,dateini,dateend,datemean,year,month,dat.varSA,fe)
}
rm(var,f,i,latmax,latmin,lf,lonmax,lonmin)

[1] "Processing file 1 from 577"
[1] "Processing file 2 from 577"
[1] "Processing file 3 from 577"
[1] "Processing file 4 from 577"
[1] "Processing file 5 from 577"
[1] "Processing file 6 from 577"
[1] "Processing file 7 from 577"
[1] "Processing file 8 from 577"
[1] "Processing file 9 from 577"
[1] "Processing file 10 from 577"
[1] "Processing file 11 from 577"
[1] "Processing file 12 from 577"
[1] "Processing file 13 from 577"
[1] "Processing file 14 from 577"
[1] "Processing file 15 from 577"
[1] "Processing file 16 from 577"
[1] "Processing file 17 from 577"
[1] "Processing file 18 from 577"
[1] "Processing file 19 from 577"
[1] "Processing file 20 from 577"
[1] "Processing file 21 from 577"
[1] "Processing file 22 from 577"
[1] "Processing file 23 from 577"
[1] "Processing file 24 from 577"
[1] "Processing file 25 from 577"
[1] "Processing file 26 from 577"
[1] "Processing file 27 from 577"
[1] "Processing file 28 from 577"
[1] "Processing file 29 from 577"
[1] "Processing file 30